In [1]:
from  typing import Final
import numpy as np
from scipy.signal import get_window


The Leaky Integrate and Fire (LIF) neuronal model

Model Parameters:

In [2]:
TOTAL_SIMULATION_TIME: Final = 50  # Total simulation time in milliseconds [T]
SIMULATION_TIME_INTERVAL: Final = 0.1  # Simulation time step in milliseconds [dt]
STIMULUS_INITIALIZATION_TIME: Final = 0  # Time at which stimulus begins in milliseconds [t_init]

RESTING_POTENTIAL_VOLTAGE: Final = -70  # Resting membrane potential in mV [V_rest]
MEMBRANE_RESISTANCE: Final = 1  # Membrane resistance in kOhm [Rm]
MEMBRANE_CAPACITANCE: Final = 5  # Capacitance of the membrane in uF [Cm]
REFRACTORY_PERIOD_TIME: Final = 1  # Refractory period duration in milliseconds [tau_ref]
SPIKE_THRESHOLD_VOLTAGE: Final = -40  # Voltage threshold for spike initiation in mV [V_th]
INPUT_CURRENT_AMPLITUDE: Final = 0.2  # Amplitude of input current in mA [I]
SPIKE_PEAK_VOLTAGE: Final = 50  # Voltage level representing a spike in mV [V_spike]

Simulation parameters:

In [3]:
# np.arange creates an array of time points for our simulation, specified by (start, stop, step).
# Starting at 0, we increment by the simulation time interval to simulate the neuron's behavior at each step.
# Stop parameter is defined as total simulation time + time interval. This ensures that the final simulation time is included in array. For example, if we did not add the time interval, and we had an interval of 3 and total time of 10, we would stop at time of 9 and never reach 10.

time_array = np.arange(0, TOTAL_SIMULATION_TIME + SIMULATION_TIME_INTERVAL,  SIMULATION_TIME_INTERVAL)
print(time_array)

[ 0.   0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  1.   1.1  1.2  1.3
  1.4  1.5  1.6  1.7  1.8  1.9  2.   2.1  2.2  2.3  2.4  2.5  2.6  2.7
  2.8  2.9  3.   3.1  3.2  3.3  3.4  3.5  3.6  3.7  3.8  3.9  4.   4.1
  4.2  4.3  4.4  4.5  4.6  4.7  4.8  4.9  5.   5.1  5.2  5.3  5.4  5.5
  5.6  5.7  5.8  5.9  6.   6.1  6.2  6.3  6.4  6.5  6.6  6.7  6.8  6.9
  7.   7.1  7.2  7.3  7.4  7.5  7.6  7.7  7.8  7.9  8.   8.1  8.2  8.3
  8.4  8.5  8.6  8.7  8.8  8.9  9.   9.1  9.2  9.3  9.4  9.5  9.6  9.7
  9.8  9.9 10.  10.1 10.2 10.3 10.4 10.5 10.6 10.7 10.8 10.9 11.  11.1
 11.2 11.3 11.4 11.5 11.6 11.7 11.8 11.9 12.  12.1 12.2 12.3 12.4 12.5
 12.6 12.7 12.8 12.9 13.  13.1 13.2 13.3 13.4 13.5 13.6 13.7 13.8 13.9
 14.  14.1 14.2 14.3 14.4 14.5 14.6 14.7 14.8 14.9 15.  15.1 15.2 15.3
 15.4 15.5 15.6 15.7 15.8 15.9 16.  16.1 16.2 16.3 16.4 16.5 16.6 16.7
 16.8 16.9 17.  17.1 17.2 17.3 17.4 17.5 17.6 17.7 17.8 17.9 18.  18.1
 18.2 18.3 18.4 18.5 18.6 18.7 18.8 18.9 19.  19.1 19.2 19.3 19.4 19.5
 19.6 

In [4]:
# For every time point in our time_array we set a membrane potential at the given time point. We set the membrane potential at every point to be equivalent to the resting potential.
# To achieve this, we first do np.ones(len(time_array)). We end up with a new array the length of time_array, except this time the entire array consists of 1. We then multiply this new array by the resting potential, making a new array where every value is just the resting potential.

# membrane potential is also referred as Vm -- to represent Voltage membrane
membrane_potential_array = np.ones(len(time_array)) * RESTING_POTENTIAL_VOLTAGE
print(membrane_potential_array)

[-70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70.
 -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70.
 -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70.
 -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70.
 -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70.
 -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70.
 -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70.
 -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70.
 -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70.
 -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70.
 -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70.
 -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70.
 -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70.
 -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70. -70.
 -70. 

In [5]:
# The membrane time constant [tau_m], refers to the speed at which a neuron's membrane potential returns to equilibrium (resting potential) after a stimuli. Thus, it also is how fast a neuron can respond to a new stimuli.
# Note that spikes are "all or nothing." They cannot be interrupted or prolonged during polarization/depolarization (firing states) or hyperpolarization (recharging state), hence why the value of [tau_m] is so important.

# To calculate:
# 1. Convert membrane resistance [Rm] from kiloohms to ohms by multiplying Rm by 1e3 (1000), as 1 kΩ equals 1000 Ω.
# 2. Convert membrane capacitance [Cm] from microfarads to farads by multiplying [Cm] by 1e-6, since 1 μF equals 1e-6 F.

# Membrane time constant [tau_m] in seconds
membrane_time_constant = MEMBRANE_RESISTANCE * 1e3 * MEMBRANE_CAPACITANCE * 1 * 1e-6
print(membrane_time_constant)

0.005


In [6]:
# We initialize an empty list, which we'll later use to record each time point which a 'spike' occurs.
# Note that a spike occurs when a neuron fires an action potential.
# In the context of the LIF model, a spike occurs when the the membrane potential [Vm] reaches or surpasses the spike threshold voltage [V_th]. 

spikes = []

Defining the stimulus:

In [8]:
input_stimulus = INPUT_CURRENT_AMPLITUDE * 1e-3 * get_window('triang', len(time_array))
print(input_stimulus)

[3.98406375e-07 1.19521912e-06 1.99203187e-06 2.78884462e-06
 3.58565737e-06 4.38247012e-06 5.17928287e-06 5.97609562e-06
 6.77290837e-06 7.56972112e-06 8.36653386e-06 9.16334661e-06
 9.96015936e-06 1.07569721e-05 1.15537849e-05 1.23505976e-05
 1.31474104e-05 1.39442231e-05 1.47410359e-05 1.55378486e-05
 1.63346614e-05 1.71314741e-05 1.79282869e-05 1.87250996e-05
 1.95219124e-05 2.03187251e-05 2.11155378e-05 2.19123506e-05
 2.27091633e-05 2.35059761e-05 2.43027888e-05 2.50996016e-05
 2.58964143e-05 2.66932271e-05 2.74900398e-05 2.82868526e-05
 2.90836653e-05 2.98804781e-05 3.06772908e-05 3.14741036e-05
 3.22709163e-05 3.30677291e-05 3.38645418e-05 3.46613546e-05
 3.54581673e-05 3.62549801e-05 3.70517928e-05 3.78486056e-05
 3.86454183e-05 3.94422311e-05 4.02390438e-05 4.10358566e-05
 4.18326693e-05 4.26294821e-05 4.34262948e-05 4.42231076e-05
 4.50199203e-05 4.58167331e-05 4.66135458e-05 4.74103586e-05
 4.82071713e-05 4.90039841e-05 4.98007968e-05 5.05976096e-05
 5.13944223e-05 5.219123